In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from skimage.feature import graycomatrix, graycoprops
from tqdm import tqdm

def extract_glcm_features(image, distances, angles):
    if image is None:
        return None

    # Mengonversi citra menjadi tipe data 8-bit unsigned integer
    image = np.uint8(image)

    # Membuat matriks GLCM
    glcm = graycomatrix(image, distances, angles, levels=256, symmetric=True, normed=True)

    # Menghitung fitur GLCM
    contrast = graycoprops(glcm, 'contrast')
    dissimilarity = graycoprops(glcm, 'dissimilarity')
    homogeneity = graycoprops(glcm, 'homogeneity')
    energy = graycoprops(glcm, 'energy')
    correlation = graycoprops(glcm, 'correlation')

    # Menggabungkan semua fitur menjadi satu vektor fitur
    features = np.concatenate([contrast.ravel(), dissimilarity.ravel(), homogeneity.ravel(),
                               energy.ravel(), correlation.ravel()])

    return features

# Path ke folder citra "Data Normal"
folder_path = 'Data_Enhance'

# Parameter GLCM (jarak dan sudut)
distances = [1, 2, 3, 4, 5]
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

# List untuk menyimpan vektor fitur GLCM
glcm_features_list = []

# Loop melalui semua citra dalam folder dengan tqdm
for filename in tqdm(os.listdir(folder_path), desc='Extracting GLCM Features'):
    # Mengimpor citra grayscale
    image_path = os.path.join(folder_path, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Mengekstraksi fitur GLCM dari citra
    glcm_features = extract_glcm_features(image, distances, angles)

    # Menambahkan vektor fitur GLCM ke list
    if glcm_features is not None:
        glcm_features_list.append(glcm_features)

# Membuat DataFrame dari vektor fitur GLCM
column_names = []
for distance in distances:
    for angle in angles:
        column_names.append(f'contrast_{distance}_{int(np.degrees(angle))}')
        column_names.append(f'dissimilarity_{distance}_{int(np.degrees(angle))}')
        column_names.append(f'homogeneity_{distance}_{int(np.degrees(angle))}')
        column_names.append(f'energy_{distance}_{int(np.degrees(angle))}')
        column_names.append(f'correlation_{distance}_{int(np.degrees(angle))}')

df = pd.DataFrame(glcm_features_list, columns=column_names)

# Mengganti nilai NaN dengan 0
df = df.fillna(0)

# Menampilkan DataFrame
print(df)


Extracting GLCM Features: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1201/1201 [01:36<00:00, 12.47it/s]

      contrast_1_0  dissimilarity_1_0  homogeneity_1_0  energy_1_0   
0       135.534069         347.452848       336.650794  342.823654  \
1        46.697733         230.791161       231.684276  233.462932   
2        37.134191         152.484781       150.010789  151.258077   
3        45.891728         259.912045       255.878348  252.501027   
4        55.980147         622.607719       612.113963  621.267102   
...            ...                ...              ...         ...   
1195     10.145221         134.304637       137.359747  133.899409   
1196     29.421324         891.732026       881.433594  892.950016   
1197     28.146446         480.889760       470.667101  478.621600   
1198     34.909865         666.328354       658.898375  665.175412   
1199     23.022794         658.007719       651.891927  659.420666   

      correlation_1_0  contrast_1_45  dissimilarity_1_45  homogeneity_1_45   
0          231.384843     347.452848          368.858493        342.823654  \
1  

In [2]:
print(df.columns)


Index(['contrast_1_0', 'dissimilarity_1_0', 'homogeneity_1_0', 'energy_1_0',
       'correlation_1_0', 'contrast_1_45', 'dissimilarity_1_45',
       'homogeneity_1_45', 'energy_1_45', 'correlation_1_45', 'contrast_1_90',
       'dissimilarity_1_90', 'homogeneity_1_90', 'energy_1_90',
       'correlation_1_90', 'contrast_1_135', 'dissimilarity_1_135',
       'homogeneity_1_135', 'energy_1_135', 'correlation_1_135',
       'contrast_2_0', 'dissimilarity_2_0', 'homogeneity_2_0', 'energy_2_0',
       'correlation_2_0', 'contrast_2_45', 'dissimilarity_2_45',
       'homogeneity_2_45', 'energy_2_45', 'correlation_2_45', 'contrast_2_90',
       'dissimilarity_2_90', 'homogeneity_2_90', 'energy_2_90',
       'correlation_2_90', 'contrast_2_135', 'dissimilarity_2_135',
       'homogeneity_2_135', 'energy_2_135', 'correlation_2_135',
       'contrast_3_0', 'dissimilarity_3_0', 'homogeneity_3_0', 'energy_3_0',
       'correlation_3_0', 'contrast_3_45', 'dissimilarity_3_45',
       'homogeneity_3_

In [3]:
df.to_csv(f'glcm_features_{folder_path}.csv', index=False)